# TRANSFORMACIÓN DE DATOS

## IMPORTACIÓN DE PAQUETES

In [42]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

## IMPORTAR LOS DATOS

### Cargar los datos

In [43]:
cat = pd.read_pickle('../../02_Datos/03_Trabajo/cat_resultado_eda.pickle')
num = pd.read_pickle('../../02_Datos/03_Trabajo/num_resultado_eda.pickle')

In [44]:
num.columns

Index(['age', 'distance_station', 'stores', 'latitude', 'longitude', 'price'], dtype='object')

### Separar la target

In [45]:
target = num.price.copy().reset_index(drop=True)
num = num.drop(columns= 'price').reset_index(drop=True)
cat = cat.reset_index(drop=True)

In [46]:
cat.columns.to_list()

['year', 'mes']

In [47]:
num.columns.to_list()

['age', 'distance_station', 'stores', 'latitude', 'longitude']

## TRANSFORMACIÓN DE CATEGÓRICAS

### One Hot Encoding

#### Variables a aplicar el One Hot Encoding

In [48]:
var_ohe = cat.columns
var_ohe

Index(['year', 'mes'], dtype='object')

#### Instanciar

In [49]:
ohe = OneHotEncoder(sparse_output= False, handle_unknown= 'ignore')
ohe

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

### Entrenar y aplicar

In [50]:
cat_ohe = ohe.fit_transform(cat[var_ohe])
cat_ohe

array([[1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

#### Guardar como dataframe

In [51]:
cat_ohe = pd.DataFrame(cat_ohe, columns= ohe.get_feature_names_out())
cat_ohe

,year_2012,year_2013,mes_1,mes_2,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,mes_10,mes_11,mes_12
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
285,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
286,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
287,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Target Encoder

#### Variables a aplicar TE

In [52]:
cat

,year,mes
0,2012,12
1,2012,12
2,2013,8
3,2013,7
4,2013,6
...,...,...
284,2013,6
285,2013,1
286,2013,4
287,2013,1


In [53]:
var_te =  ['year', 'mes']

#### Instanciar

In [54]:
te = TargetEncoder(min_samples_leaf= 100, return_df= False)

#### Entrenar y aplicar

In [55]:
cat_te = te.fit_transform(cat[var_te], y= target)

#### Guardar como dataset

In [56]:
nombre_var_te = [variable + '_te' for variable in var_te] 

In [57]:
# Guardar como dataframe

cat_te = pd.DataFrame(cat_te, columns= nombre_var_te)

In [58]:
cat_te.mes_te.value_counts()

37.679898    42
37.673846    37
37.669208    36
37.668427    28
37.667454    23
37.672632    21
37.670426    21
37.671434    20
37.670257    20
37.671432    15
37.671590    15
37.670724    11
Name: mes_te, dtype: int64

## TRANSFORMACIÓN DE NUMÉRICAS

Las variables numéricas tienen que ser rescaladas.

## UNIFICAR DATASET TRANSFORMADOS

In [59]:
df = pd.concat([cat_te, num], axis= 1)
df

,year_te,mes_te,age,distance_station,stores,latitude,longitude
0,37.402318,37.668427,32.0,84.87882,10,24.98298,121.54024
1,37.402318,37.668427,19.5,306.59470,9,24.98034,121.53951
2,38.533477,37.671432,13.3,561.98450,5,24.98746,121.54391
3,38.533477,37.669208,13.3,561.98450,5,24.98746,121.54391
4,38.533477,37.673846,20.3,287.60250,6,24.98042,121.54228
...,...,...,...,...,...,...,...
284,38.533477,37.673846,18.5,2175.74400,3,24.96330,121.51243
285,38.533477,37.667454,13.7,4082.01500,0,24.94155,121.50381
286,38.533477,37.672632,18.8,390.96960,7,24.97923,121.53986
287,38.533477,37.667454,8.1,104.81010,5,24.96674,121.54067


## REESCALAR VARIABLES

### MIN-MAX

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year_te           289 non-null    float64
 1   mes_te            289 non-null    float64
 2   age               289 non-null    float64
 3   distance_station  289 non-null    float64
 4   stores            289 non-null    int64  
 5   latitude          289 non-null    float64
 6   longitude         289 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 15.9 KB


### Variables a reescalar 

In [61]:
var_mms = df.columns
var_mms

Index(['year_te', 'mes_te', 'age', 'distance_station', 'stores', 'latitude',
       'longitude'],
      dtype='object')

### Instanciar

In [62]:
mms = MinMaxScaler()

### Entrenar y aplicar

In [63]:
df_mms = mms.fit_transform(df[var_mms])

### Guardar como dataframe

#### Añadir sufijos a los nombre

In [64]:
nombre_mms = [variable + '_mms' for variable in var_mms]
nombre_mms

['year_te_mms',
 'mes_te_mms',
 'age_mms',
 'distance_station_mms',
 'stores_mms',
 'latitude_mms',
 'longitude_mms']

#### Guardar como dataframe

In [65]:
df_mms = pd.DataFrame(df_mms, columns= nombre_mms)
df_mms

,year_te_mms,mes_te_mms,age_mms,distance_station_mms,stores_mms,latitude_mms,longitude_mms
0,0.0,0.078202,0.749415,0.005549,1.0,0.616941,0.714301
1,0.0,0.078202,0.456674,0.040484,0.9,0.584949,0.706289
2,1.0,0.319701,0.311475,0.080724,0.5,0.671231,0.754582
3,1.0,0.140970,0.311475,0.080724,0.5,0.671231,0.754582
4,1.0,0.513659,0.475410,0.037491,0.6,0.585919,0.736692
...,...,...,...,...,...,...,...
284,1.0,0.513659,0.433255,0.334994,0.3,0.378454,0.409066
285,1.0,0.000000,0.320843,0.635354,0.0,0.114881,0.314455
286,1.0,0.416116,0.440281,0.053778,0.7,0.571498,0.710131
287,1.0,0.000000,0.189696,0.008690,0.5,0.420141,0.719021


## UNIFICAR DATASETS RESCALADOS

#### Lista de datasets a unificar

In [66]:
df_unir = [cat_ohe, df_mms, target]

#### Unificar Datasets en tablón Analítico

In [67]:
df_tablon = pd.concat(df_unir, axis=1)
df_tablon

,year_2012,year_2013,mes_1,mes_2,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,...,mes_11,mes_12,year_te_mms,mes_te_mms,age_mms,distance_station_mms,stores_mms,latitude_mms,longitude_mms,price
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.078202,0.749415,0.005549,1.0,0.616941,0.714301,37.9
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.078202,0.456674,0.040484,0.9,0.584949,0.706289,42.2
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.319701,0.311475,0.080724,0.5,0.671231,0.754582,47.3
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.140970,0.311475,0.080724,0.5,0.671231,0.754582,54.8
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.513659,0.475410,0.037491,0.6,0.585919,0.736692,46.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.513659,0.433255,0.334994,0.3,0.378454,0.409066,28.1
285,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.000000,0.320843,0.635354,0.0,0.114881,0.314455,15.4
286,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.416116,0.440281,0.053778,0.7,0.571498,0.710131,40.6
287,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.000000,0.189696,0.008690,0.5,0.420141,0.719021,52.5


## GUARDAR DATASETS TRANSFORMACIÓN DE DATOS

#### Guardamos el tablón analítico en formato pickle

In [68]:
df_tablon.to_pickle('../../02_Datos/03_Trabajo/df_tablon.pickle')